In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os
simp_path = '.env'
abs_path = os.path.abspath(simp_path)
CREDENTIAL={'EMAIL':'','PASSWORD':''}
with open(abs_path,"r") as archivo:
    for linea in archivo:        
        if (str(linea.strip('\n')).split('='))[0] == "EMAIL":
            CREDENTIAL["EMAIL"]=(str(linea.strip('\n')).split('='))[1]
        if (str(linea.strip('\n')).split('='))[0] == "PASSWORD":
            CREDENTIAL["PASSWORD"]=(str(linea.strip('\n')).split('='))[1]
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#abrir formulario


driver.get('https://sivireno.undc.edu.pe/index_login.php')
driver.find_element(by=By.XPATH, value='/html/body/div[1]/div/div/form/div[1]/a').click()
driver.find_element(by=By.XPATH, value='//*[@id="identifierId"]').send_keys(CREDENTIAL["EMAIL"])
driver.find_element(by=By.XPATH, value='//*[@id="identifierNext"]/div/button/span').click()
for j in range(10):
    try:
        driver.find_element(by=By.XPATH, value='//*[@id="password"]/div[1]/div/div[1]/input').send_keys(CREDENTIAL["PASSWORD"])
        driver.find_element(by=By.XPATH, value='//*[@id="passwordNext"]/div/button/span').click()
        break
    except:
        time.sleep(1)

In [7]:
##SCRIPT EXTRAER TODOS LOS LINK DE CARGA LECTIVA Y NO LECTIVA
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
import re
from selenium.common.exceptions import NoSuchElementException
import time
asig = []
ciclo = []
t = []
s = []
c = []
docen = []
cl = []
cnl = []
av=[]
sila=[]
baseURL="https://sivireno.undc.edu.pe/"
##la cantidad que te muestra el sivireno
cantlist=100

###########################################CREACION DE METODOS#################################################################

def addLinkExcel():
    df = pd.DataFrame({ 'ASIGNATURA': asig,'CICLO': ciclo,'TURNO': t,'SECCION': s,'CANT': c,'DOCENTE': docen,'CL': cl,'CNL': cnl,'AV':av,'SILABUS':sila})
    df = df[['ASIGNATURA', 'CICLO','TURNO','SECCION','CANT','DOCENTE','CL','CNL','AV','SILABUS']]
    writer = pd.ExcelWriter('EXCEL_GENERADOS/CREATE_LIST_CURSOS_AGRONOMIA'+time.strftime("%Y%m%d-%H-%M-%S")+'.xlsx')
    df.to_excel(writer, 'Sheet', index=False)
    #writer.save()
    writer.close()
    print("TERMINO PROCESO")
###########################################CREACION DE METODOS#################################################################
try:
    for j in range(int(cantlist)):
        #SOLO PARA LA MODALIDAD DEL CURSO EN M
        if driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[3]').text =="M":
            asig.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[4]').text)
            ciclo.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[5]').text)
            t.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[6]').text)
            s.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[7]').text)
            c.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[8]').text)
            docen.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[9]').text)
            cl.append(baseURL+driver.execute_script('return document.querySelector("#app > div.table-responsive > table > tbody  > tr:nth-child('+str(j+1)+') > td:nth-child(10) > a:nth-child(1)").getAttribute("href");'))
            cnl.append(baseURL+driver.execute_script('return document.querySelector("#app > div.table-responsive > table > tbody > tr:nth-child('+str(j+1)+') > td:nth-child(10) > a:nth-child(2)").getAttribute("href");'))
            av.append(driver.execute_script('return document.querySelector("#app > div.table-responsive > table > tbody  > tr:nth-child('+str(j+1)+') > td:nth-child(13) > a").getAttribute("href");'))
            sila.append(driver.execute_script('let element = document.querySelector("#app > div.table-responsive > table > tbody > tr:nth-child(' + str(j+1) + ') > td:nth-child(11) > a");return element ? element.getAttribute("href") : null;'))
    addLinkExcel()
except:
    addLinkExcel()


ValueError: All arrays must be of the same length